First some libraries are imported. Of course, we use [Text-Fabric with the BHSA data](https://etcbc.github.io/bhsa) which you can access with Python 3 and as framework for the Neural Network we use [Keras](https://keras.io).


In [3]:
import collections

import pandas as pd
import numpy as np

from sklearn.utils import shuffle
from statistics import mode

from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam #does not work: from keras.optimizers import Adam

In [4]:
!pip install text-fabric
!pip install --upgrade text-fabric

     |████████████████████████████████| 7.3 MB 2.8 MB/s 
     |████████████████████████████████| 596 kB 42.5 MB/s 
     |████████████████████████████████| 68 kB 6.9 MB/s 
     |████████████████████████████████| 291 kB 51.2 MB/s 
     |████████████████████████████████| 961 kB 36.1 MB/s 
     |████████████████████████████████| 117 kB 51.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [5]:
from tf.app import use
from tf.fabric import Fabric
A = use('bhsa', hoist=globals())

A.displaySetup(extraFeatures='g_cons') #this can be removed. consonantal reprsentation of a word in the manuscript. lexeme is dictionary form

The requested TF-app is not available offline
	~/text-fabric-data/annotation/app-bhsa/code not found
rate limit is 60 requests per hour, with 60 left for this hour
To increase the rate,see https://annotation.github.io/text-fabric/tf/advanced/repo.html/
	connecting to online GitHub repo annotation/app-bhsa ... connected
	code/__init__.py...downloaded
	code/app.py...downloaded
	code/config.yaml...downloaded
	code/static...directory
		code/static/logo.png...downloaded
	OK


The requested data is not available offline
	~/text-fabric-data/etcbc/bhsa/tf not found
rate limit is 60 requests per hour, with 44 left for this hour
To increase the rate,see https://annotation.github.io/text-fabric/tf/advanced/repo.html/
	connecting to online GitHub repo etcbc/bhsa ... connected
	downloading https://github.com/ETCBC/bhsa/releases/download/v1.7.3/tf-2021.zip ... 
	unzipping ... 
	saving data


The requested data is not available offline
	~/text-fabric-data/etcbc/phono/tf not found
rate limit is 60 requests per hour, with 35 left for this hour
To increase the rate,see https://annotation.github.io/text-fabric/tf/advanced/repo.html/
	connecting to online GitHub repo etcbc/phono ... connected
	downloading https://github.com/ETCBC/phono/releases/download/v2.1/tf-2021.zip ... 
	unzipping ... 
	saving data


The requested data is not available offline
	~/text-fabric-data/etcbc/parallels/tf not found
rate limit is 60 requests per hour, with 26 left for this hour
To increase the rate,see https://annotation.github.io/text-fabric/tf/advanced/repo.html/
	connecting to online GitHub repo etcbc/parallels ... connected
	downloading https://github.com/ETCBC/parallels/releases/download/v2.1/tf-2021.zip ... 
	unzipping ... 
	saving data


This is Text-Fabric 9.1.13
Api reference : https://annotation.github.io/text-fabric/tf/cheatsheet.html

122 features found and 0 ignored
   |     3.59s T otype                from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |       42s T oslots               from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     0.00s T book@ru              from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     3.97s T lex_utf8             from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     0.00s T book@syc             from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     4.00s T voc_lex_utf8         from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     0.00s T book@es              from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     0.00s T book@he              from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     6.14s T phono                from ~/text-fabric-data/etcbc/phono/tf/2021
   |     0.00s T book@zh              from ~/text-fabric-data/etcbc/bhsa/tf/2021
   |     4.74s T phono_trailer        from ~/text-fa

In [4]:
F.otype.all

('book',
 'chapter',
 'lex',
 'verse',
 'half_verse',
 'sentence',
 'sentence_atom',
 'clause',
 'clause_atom',
 'phrase',
 'phrase_atom',
 'subphrase',
 'word')

In [ ]:
bhsBooks = [F.book.v(n) for n in F.otype.s("book")]
print(F.book.v("Genesis"))
print(F.otype.s("book"))
print(bhsBooks)

None
range(426591, 426630)
['Genesis', 'Exodus', 'Leviticus', 'Numeri', 'Deuteronomium', 'Josua', 'Judices', 'Samuel_I', 'Samuel_II', 'Reges_I', 'Reges_II', 'Jesaia', 'Jeremia', 'Ezechiel', 'Hosea', 'Joel', 'Amos', 'Obadia', 'Jona', 'Micha', 'Nahum', 'Habakuk', 'Zephania', 'Haggai', 'Sacharia', 'Maleachi', 'Psalmi', 'Iob', 'Proverbia', 'Ruth', 'Canticum', 'Ecclesiastes', 'Threni', 'Esther', 'Daniel', 'Esra', 'Nehemia', 'Chronica_I', 'Chronica_II']


In [ ]:
F.otype.s("chapter")

range(426630, 427559)

In [ ]:
F.otype.s("book")

range(426591, 426630)

In [ ]:
F.book.v("Genesis")

In [ ]:
print(L.d(5, 'word'))

()


In [10]:
for bo in F.otype.s("book"): #books of the hebrew Bible
  print(bo, F.book.v(bo)) #these are nodes
  if (F.book.v(bo) == 'Jona'):
    #all the words in Jonah
    words = L.d(bo, 'word') #all the words in Jonah; moves fro one type of object to nother type of object: book chapter word sentenc clause 
    print(words)
    #print all lexemes in Jonah
    print([F.lex.v(w) for w in words])
    print([F.g_cons.v(w) for w in words])
    #for every word, look at clause
    

426591 Genesis
426592 Exodus
426593 Leviticus
426594 Numeri
426595 Deuteronomium
426596 Josua
426597 Judices
426598 Samuel_I
426599 Samuel_II
426600 Reges_I
426601 Reges_II
426602 Jesaia
426603 Jeremia
426604 Ezechiel
426605 Hosea
426606 Joel
426607 Amos
426608 Obadia
426609 Jona
(298558, 298559, 298560, 298561, 298562, 298563, 298564, 298565, 298566, 298567, 298568, 298569, 298570, 298571, 298572, 298573, 298574, 298575, 298576, 298577, 298578, 298579, 298580, 298581, 298582, 298583, 298584, 298585, 298586, 298587, 298588, 298589, 298590, 298591, 298592, 298593, 298594, 298595, 298596, 298597, 298598, 298599, 298600, 298601, 298602, 298603, 298604, 298605, 298606, 298607, 298608, 298609, 298610, 298611, 298612, 298613, 298614, 298615, 298616, 298617, 298618, 298619, 298620, 298621, 298622, 298623, 298624, 298625, 298626, 298627, 298628, 298629, 298630, 298631, 298632, 298633, 298634, 298635, 298636, 298637, 298638, 298639, 298640, 298641, 298642, 298643, 298644, 298645, 298646, 298647

In [15]:
jonah_words = [298558, 298559, 298560, 298561, 298562, 298563, 298564, 298565, 298566, 298567, 298568, 298569, 298570]
jonah_set = set()

for word in jonah_words:
  clause = L.u(word, 'clause')[0] #oth pos is node in the clause 
  jonah_set.add(clause)

for clause in jonah_set:
  print(T.text(clause))
  phrases = L.d(clause, 'phrase')
  print([F.function.v(p) for p in phrases])

וַֽיְהִי֙ דְּבַר־יְהוָ֔ה אֶל־יֹונָ֥ה בֶן־אֲמִתַּ֖י 
['Conj', 'Pred', 'Subj', 'Cmpl']
לֵאמֹֽר׃ 
['Pred']
ק֠וּם 
['Pred']
לֵ֧ךְ אֶל־נִֽינְוֵ֛ה הָעִ֥יר הַגְּדֹולָ֖ה 
['Pred', 'Cmpl']


In [16]:
function_set = set()
for ph in F.otype.s("phrase"):
  function_set.add(F.function.v(ph))
print(function_set) #all unique values for phrase functions in the bhsa



{'PrAd', 'Cmpl', 'Intj', 'PreO', 'Ques', 'ExsS', 'Subj', 'PreC', 'Modi', 'IntS', 'Time', 'PrcS', 'Adju', 'EPPr', 'ModS', 'Loca', 'PreS', 'Supp', 'Conj', 'Nega', 'Objc', 'PtcO', 'Exst', 'Pred', 'NCoS', 'Frnt', 'Voct', 'Rela', 'NCop'}


In [21]:
subject_dict = {}

#print all the subjects in Hebrew where they occur (eg. Genesis 1) elohim
for ph in F.otype.s("phrase"):
  if F.function.v(ph) == 'Subj':
    bo,ch,ve = T.sectionFromNode(ph) #unpack tuple by assigning each value to vars bo, ch, ve
    subject_text = ' '.join([F.g_cons.v(w) for w in L.d(ph ,'word')])
    info_list = [bo,ch,ve,subject_text]
    print(info_list)
    subject_dict[ph] = info_list
    #print([F.g_cons.v(w) for w in L.d(ph ,'word')]) # print([F.g_cons.v(w) for w in words]); #book, chapter, verse



Streaming output truncated to the last 5000 lines.
['Song_of_songs', 8, 5, '>MK']
['Song_of_songs', 8, 6, '>HBH']
['Song_of_songs', 8, 6, 'QN>H']
['Song_of_songs', 8, 6, 'RCPJH']
['Song_of_songs', 8, 7, 'MJM RBJM']
['Song_of_songs', 8, 7, 'NHRWT']
['Song_of_songs', 8, 7, '>JC']
['Song_of_songs', 8, 8, '>XWT QVNH']
['Song_of_songs', 8, 8, 'CDJM']
['Song_of_songs', 8, 9, 'HJ>']
['Song_of_songs', 8, 9, 'HJ>']
['Song_of_songs', 8, 10, '>NJ']
['Song_of_songs', 8, 10, 'CDJ']
['Song_of_songs', 8, 11, 'KRM']
['Song_of_songs', 8, 11, '>JC']
['Song_of_songs', 8, 12, 'KRMJ']
['Song_of_songs', 8, 12, 'H >LP']
['Song_of_songs', 8, 12, 'M>TJM']
['Song_of_songs', 8, 13, 'XBRJM']
['Ecclesiastes', 1, 1, 'DBRJ QHLT BN DWD MLK B JRWCLM']
['Ecclesiastes', 1, 2, 'QHLT']
['Ecclesiastes', 1, 2, 'H KL']
['Ecclesiastes', 1, 3, 'JTRWN L  >DM B KL <MLW']
['Ecclesiastes', 1, 4, 'DWR']
['Ecclesiastes', 1, 4, 'DWR']
['Ecclesiastes', 1, 4, 'H >RY']
['Ecclesiastes', 1, 5, 'H CMC']
['Ecclesiastes', 1, 5, 'H CMC']
['Ec

In [27]:
from pprint import pprint
import pandas as pd
subject_df = pd.DataFrame(subject_dict).T 
subject_df.columns = ["book", "chapter", "verse", "data"]
subject_df.head()
subject_df.to_csv('subject.csv') #specify path name
#pprint(subject_dict) #in general these are phrases in order; guarantee that words in order
